# Single Particle performance plots


   * [#$p_{T}$ response](#$p_{T}$-response)


In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load ../drawingTools



In [11]:
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v76t'


def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary


# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'V8'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'V8')
    ]


samples_ele_V9 = [
    Sample('all_flat5to80_PU0', 'PU0', 'v78', 'V9'),
    Sample('all_flat5to80_PU200', 'PU200', 'v78', 'V9')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200', 'PU200', version, 'photon')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

all_tpsets = {'DEF': 'dRC3d',
              'DEFCalib': 'NNDR Calib v1',
              'DEFNC': 'dRC3d + new Th',
              'HMvDR': 'HistoMaxC3d + dR(layer)',
              'HMvDRNC0': 'HMC3d+dR(layer)+NC0',
              'HMvDRNC1': 'HMC3d+dR(layer)+NC1',
              'EG': 'EG',
              'TkEG': 'TkEG',
              'TkEle': 'TkEle',
              'TkIsoEle': 'TkIsoEle',
              'L1Trk': 'L1Track'}


Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU0_v76t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU200_v76t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_pion_flat2to100_PU0_v76t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu0_v76t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v76t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu200_v76t.root does not exist


In [4]:
# %load settings.py
# === samples =====================================================
samples = []

samples = samples_ele_V9
samples += samples_ele_V8


sample = 'V9'
# === TP ==========================================================
tps = ['EG']

# === TP selections ================================================
tp_select = {}
tp_select['DEF'] = ['Em', 'all']
tp_select['DEFNC'] = tp_select['DEF']
tp_select['HMvDR'] = tp_select['DEF']
tp_select['HMvDRNC0'] = tp_select['DEF']
tp_select['HMvDRNC1'] = tp_select['DEF']


tp_select['EG'] = ['EGq2', 'EGq3']

tp_select['TkEG'] = ['EGq2', 'EGq3']
tp_select['TkEle'] = ['EGq2', 'EGq3']

tp_select['TkEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['TkIsoEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['L1Trk'] = ['all', 'Pt2', 'Pt10']

# ==== GEN selections ===============================================

gen_select = ['GENEtaBC', 'GENEtaD']
# gen_select = ['GEN', 'GENEtaA', 'GENEtaB', 'GENEtaC', 'GENEtaD', 'GENEtaE',
#               'GENEtaAB', 'GENEtaABC', 'GENEtaBC', 'GENEtaBCD', 'GENEtaBCDE' ]

# tp_select['TkEG'] = ['EGq2EtaBC', 'EGq3EtaBC', 'EGq2EtaBCM2', 'EGq3EtaBCM2', 'EGq2EtaBCM2s', 'EGq3EtaBCM2s','EGq2EtaBCM3', 'EGq3EtaBCM3', 'EGq2EtaBCM3s', 'EGq3EtaBCM3s','EGq2EtaBCM4', 'EGq3EtaBCM4']


# ==== adapt the plot retrieval
tpsets = {}
tpset_selections = {}
gen_selections = {}

for tp in tps:
    tpsets[tp] = all_tpsets[tp]

if 'DEF' in tps or 'HMvDR' in tps:
    tpset_selections.update(get_label_dict(selections.tp_match_selections))
if 'EG' in tps:
    tpset_selections.update(get_label_dict(selections.eg_pt_selections))
if 'TkEG' in tps:
    tpset_selections.update(get_label_dict(selections.tkeg_pt_selections))
if 'TkEle' in tps or 'TkIsoEle' in tps:
    tpset_selections.update(get_label_dict(selections.tkisoeg_pt_selections))
if 'L1Trk' in tps:
    tpset_selections.update(get_label_dict(selections.tracks_selections))


gen_selections.update(get_label_dict(selections.gen_part_selections))
gen_selections.update({'nomatch': ''})

print '--- samples:'
print samples
print '--- TPs: '
print tps
print '--- TP selections:'
print tp_select
print '--- GEN selections:'
print gen_select


--- samples:
[<drawingTools.Sample instance at 0x1136fefc8>, <drawingTools.Sample instance at 0x114209638>, <drawingTools.Sample instance at 0x114200c20>, <drawingTools.Sample instance at 0x114209680>]
--- TPs: 
['EG']
--- TP selections:
{'TkEG': ['EGq2', 'EGq3'], 'TkEle': ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3'], 'HMvDRNC0': ['Em', 'all'], 'HMvDRNC1': ['Em', 'all'], 'EG': ['EGq2', 'EGq3'], 'DEFNC': ['Em', 'all'], 'L1Trk': ['all', 'Pt2', 'Pt10'], 'TkIsoEle': ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3'], 'HMvDR': ['Em', 'all'], 'DEF': ['Em', 'all']}
--- GEN selections:
['GENEtaBC', 'GENEtaD']


In [12]:
tpset_selections

{'EGq2': 'q2',
 'EGq2Pt10': 'q2, p_{T}^{L1}>=10GeV',
 'EGq2Pt20': 'q2, p_{T}^{L1}>=20GeV',
 'EGq2Pt30': 'q2, p_{T}^{L1}>=30GeV',
 'EGq3': 'q3',
 'EGq3Pt10': 'q3, p_{T}^{L1}>=10GeV',
 'EGq3Pt20': 'q3, p_{T}^{L1}>=20GeV',
 'EGq3Pt30': 'q3, p_{T}^{L1}>=30GeV'}

In [13]:
gen_selections

{'GEN': '',
 'GENEtaB': '1.52 < |#eta^{GEN}| <= 1.7',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaC': '1.7 < |#eta^{GEN}| <= 2.4',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV',
 'nomatch': ''}

## Resolution Studies
### 3D cluster: Pt resolution

In [14]:
files

{}

In [15]:
%%time


hplot = HPlot(samples, tpsets, tpset_selections, gen_selections)


hplot.cache_histo(classtype=histos.EGResoHistos, 
                  samples=samples,
                  pus=[],
                  tps=tpsets,
                  tp_sels=tpset_selections,
                  gen_sels=gen_selections)


print dir(samples[0])
print samples[0].histo_filename
# for sample in samples:
#     print sample
#     for tp in tpsets:
#         for tp_sel in tpset_selections:
#             for gen_sel in gen_selections:
#                 print sample, tp, tp_sel, gen_sel
#                 hplot.data = hplot.data.append({'sample': 'ele',
#                                                 'pu': sample.label,
#                                                 'tp': tp,
#                                                 'tp_sel': tp_sel,
#                                                 'gen_sel': gen_sel,
#                                                 'classtype': histos.HistoSetReso,
#                                                 'histo': histos.HistoSetReso('{}_{}_{}'.format(tp, tp_sel, gen_sel), sample.histo_file)}
#                                               , ignore_index=True)


<drawingTools.Sample instance at 0x1136fefc8>
<drawingTools.Sample instance at 0x1136fefc8> EG EGq3Pt10 GENPt40
<drawingTools.Sample instance at 0x1136fefc8> EG EGq3Pt10 GENEtaC
<drawingTools.Sample instance at 0x1136fefc8> EG EGq3Pt10 GENEtaB
<drawingTools.Sample instance at 0x1136fefc8> EG EGq3Pt10 GENEtaD
<drawingTools.Sample instance at 0x1136fefc8> EG EGq3Pt10 GENEtaBC
<drawingTools.Sample instance at 0x1136fefc8> EG EGq3Pt10 GENPt30
<drawingTools.Sample instance at 0x1136fefc8> EG EGq3Pt10 GENPt20
<drawingTools.Sample instance at 0x1136fefc8> EG EGq3Pt10 GENEtaBCD
<drawingTools.Sample instance at 0x1136fefc8> EG EGq3Pt10 GENPt10
<drawingTools.Sample instance at 0x1136fefc8> EG EGq3Pt10 nomatch
<drawingTools.Sample instance at 0x1136fefc8> EG EGq3Pt10 GEN
<drawingTools.Sample instance at 0x1136fefc8> EG EGq3 GENPt40
<drawingTools.Sample instance at 0x1136fefc8> EG EGq3 GENEtaC
<drawingTools.Sample instance at 0x1136fefc8> EG EGq3 GENEtaB
<drawingTools.Sample instance at 0x1136fefc

<drawingTools.Sample instance at 0x114209638> EG EGq3Pt30 nomatch
<drawingTools.Sample instance at 0x114209638> EG EGq3Pt30 GEN
<drawingTools.Sample instance at 0x114209638> EG EGq2Pt20 GENPt40
<drawingTools.Sample instance at 0x114209638> EG EGq2Pt20 GENEtaC
<drawingTools.Sample instance at 0x114209638> EG EGq2Pt20 GENEtaB
<drawingTools.Sample instance at 0x114209638> EG EGq2Pt20 GENEtaD
<drawingTools.Sample instance at 0x114209638> EG EGq2Pt20 GENEtaBC
<drawingTools.Sample instance at 0x114209638> EG EGq2Pt20 GENPt30
<drawingTools.Sample instance at 0x114209638> EG EGq2Pt20 GENPt20
<drawingTools.Sample instance at 0x114209638> EG EGq2Pt20 GENEtaBCD
<drawingTools.Sample instance at 0x114209638> EG EGq2Pt20 GENPt10
<drawingTools.Sample instance at 0x114209638> EG EGq2Pt20 nomatch
<drawingTools.Sample instance at 0x114209638> EG EGq2Pt20 GEN
<drawingTools.Sample instance at 0x114200c20>
<drawingTools.Sample instance at 0x114200c20> EG EGq3Pt10 GENPt40
<drawingTools.Sample instance at 0x

<drawingTools.Sample instance at 0x114209680> EG EGq2 GENPt30
<drawingTools.Sample instance at 0x114209680> EG EGq2 GENPt20
<drawingTools.Sample instance at 0x114209680> EG EGq2 GENEtaBCD
<drawingTools.Sample instance at 0x114209680> EG EGq2 GENPt10
<drawingTools.Sample instance at 0x114209680> EG EGq2 nomatch
<drawingTools.Sample instance at 0x114209680> EG EGq2 GEN
<drawingTools.Sample instance at 0x114209680> EG EGq2Pt10 GENPt40
<drawingTools.Sample instance at 0x114209680> EG EGq2Pt10 GENEtaC
<drawingTools.Sample instance at 0x114209680> EG EGq2Pt10 GENEtaB
<drawingTools.Sample instance at 0x114209680> EG EGq2Pt10 GENEtaD
<drawingTools.Sample instance at 0x114209680> EG EGq2Pt10 GENEtaBC
<drawingTools.Sample instance at 0x114209680> EG EGq2Pt10 GENPt30
<drawingTools.Sample instance at 0x114209680> EG EGq2Pt10 GENPt20
<drawingTools.Sample instance at 0x114209680> EG EGq2Pt10 GENEtaBCD
<drawingTools.Sample instance at 0x114209680> EG EGq2Pt10 GENPt10
<drawingTools.Sample instance at 

In [16]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,V9,PU0,EG,EGq3Pt10,GENPt40,python.l1THistos.EGResoHistos,<drawingTools.HProxy instance at 0x103160200>
1,V9,PU0,EG,EGq3Pt10,GENEtaC,python.l1THistos.EGResoHistos,<drawingTools.HProxy instance at 0x114209d40>
2,V9,PU0,EG,EGq3Pt10,GENEtaB,python.l1THistos.EGResoHistos,<drawingTools.HProxy instance at 0x1145bce60>
3,V9,PU0,EG,EGq3Pt10,GENEtaD,python.l1THistos.EGResoHistos,<drawingTools.HProxy instance at 0x11445df38>
4,V9,PU0,EG,EGq3Pt10,GENEtaBC,python.l1THistos.EGResoHistos,<drawingTools.HProxy instance at 0x1144415f0>
5,V9,PU0,EG,EGq3Pt10,GENPt30,python.l1THistos.EGResoHistos,<drawingTools.HProxy instance at 0x1144411b8>
6,V9,PU0,EG,EGq3Pt10,GENPt20,python.l1THistos.EGResoHistos,<drawingTools.HProxy instance at 0x1145cc128>
7,V9,PU0,EG,EGq3Pt10,GENEtaBCD,python.l1THistos.EGResoHistos,<drawingTools.HProxy instance at 0x114209cb0>
8,V9,PU0,EG,EGq3Pt10,GENPt10,python.l1THistos.EGResoHistos,<drawingTools.HProxy instance at 0x11445d518>
9,V9,PU0,EG,EGq3Pt10,nomatch,python.l1THistos.EGResoHistos,<drawingTools.HProxy instance at 0x114441bd8>


In [13]:
hplot.labels_dict

{'EG': 'EG',
 'EGq2': 'q2',
 'EGq2Pt10': 'q2, p_{T}^{L1}>=10GeV',
 'EGq2Pt20': 'q2, p_{T}^{L1}>=20GeV',
 'EGq2Pt30': 'q2, p_{T}^{L1}>=30GeV',
 'EGq3': 'q3',
 'EGq3Pt10': 'q3, p_{T}^{L1}>=10GeV',
 'EGq3Pt20': 'q3, p_{T}^{L1}>=20GeV',
 'EGq3Pt30': 'q3, p_{T}^{L1}>=30GeV',
 'GEN': '',
 'GENEtaB': '1.52 < |#eta^{GEN}| <= 1.7',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaC': '1.7 < |#eta^{GEN}| <= 2.4',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV',
 'PU0': 'PU0',
 'PU200': 'PU200',
 'ele': 'ele',
 'nomatch': ''}

In [14]:
hfd = hplot.get_histo(histos.EGResoHistos, 'ele', ['PU0'], ['EG'], 'EGq2', 'GEN', True)

print hfd[0]
print hfd[1]
print hfd[2]
print dir(hfd[0][0])

# print hfd[0][0].h_ptRespVeta_mdn()

   sample   pu  tp tp_sel gen_sel                      classtype  \
32    ele  PU0  EG   EGq2     GEN  python.l1THistos.EGResoHistos   

                                        histo  
32  <__main__.HProxy instance at 0x121664f38>  
[<EGResoHistos EG_EGq2_GEN>]
['']
PU0, EG, q2
['GraphBuilder', '__doc__', '__init__', '__module__', '__repr__', 'fill', 'h_drRes', 'h_etaRes', 'h_phiRes', 'h_ptResVpt', 'h_ptResVpt_avg', 'h_ptResVpt_gmean', 'h_ptResVpt_mdn', 'h_ptResVpt_sigma', 'h_ptResp', 'h_ptRespVeta', 'h_ptRespVeta_avg', 'h_ptRespVeta_gmean', 'h_ptRespVeta_mdn', 'h_ptRespVeta_sigma', 'h_ptRespVpt', 'h_ptRespVpt_avg', 'h_ptRespVpt_gmean', 'h_ptRespVpt_mdn', 'h_ptRespVpt_sigma', 'name_', 'write']


In [ ]:
print hfd[2]


#### Matching to the highes pT cluster in DR 0.2 from GEN

In [ ]:
# ROOT.gStyle.SetPadTopMargin(0.05)
# ROOT.gStyle.SetPadTopMargin(0.13)
ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.30)

ROOT.gStyle.SetCanvasBorderMode(0)
ROOT.gStyle.SetCanvasColor(0)
ROOT.gStyle.SetCanvasDefH(600)
ROOT.gStyle.SetCanvasDefW(800)


## Angular resolution

In [ ]:

for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU0', 'PU200'], tp, tp_sel, gen_sel, True)            
            draw([his.h_phiRes for his in hsets], labels, norm=True, text=text)


In [ ]:

for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_etaRes for his in hsets], labels, norm=True, text=text)


In [ ]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_drRes for his in hsets], labels, norm=True, text=text)


## $p_{T}$ response

In [17]:
for tp in tps:
    for tp_sel in tp_select[tp]:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_ptResp for his in hsets], labels, norm=True, text=text, options='hist')


In [10]:

for tp in tps:
    for pu in ['PU0', 'PU200']:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, 'V8', pu, tp, ['EGq2', 'EGq3'], gen_sel)            
            draw([his.h_ptResp for his in hsets], labels, norm=True, text=text, options='hist')


In [25]:

for tp in tps:
    for pu in ['PU0', 'PU200']:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, sample, pu, tp, ['EGq2', 'EGq3'], gen_sel)            
            draw([his.h_ptResp for his in hsets], labels, norm=True, text=text, options='hist')


In [ ]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_ptRespVpt for his in hsets], labels, options="COLZ", do_profile=False, text=text)


In [ ]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_ptRespVpt_mdn() for his in hsets], labels, options="COLZ", do_profile=False, text=text, min_y=0, max_y=3)


In [ ]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in ['GENEtaC']:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_ptRespVpt_gmean() for his in hsets], labels, options="COLZ", do_profile=False, text=text, min_y=0, max_y=3)


In [ ]:
for tp in tps:
    for pu in ['PU0', 'PU200']:
    #for tp_sel in tp_select:
        for gen_sel in ['GENEtaC']:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], pu, tp, ['EGq2', 'EGq3'], gen_sel)            
            draw([his.h_ptRespVpt_sigma() for his in hsets], labels, options="COLZ", do_profile=False, text=text, min_y=0, max_y=3)


In [ ]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, ['ele'], ['PU200'], tp, tp_sel, gen_sel)            
            plots = []
            plots.append(hsets[0].h_ptRespVpt.ProjectionY(uuid.uuid4().hex[:6], 21, 22))
            plots.append(hsets[0].h_ptRespVpt.ProjectionY(uuid.uuid4().hex[:6], 22, 23))
            drawSame(plots, ['A', 'B'], text=text)
            #drawSeveral([his.h_ptRespVpt for his in hsets], labels, options="COLZ", do_profile=False, text=text)


In [ ]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in ['GEN']:
            hsets, labels, text = hplot.get_histo(histos.EGResoHistos, 'ele', ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            print labels
            print text
            drawSeveral([his.h_ptRespVeta for his in hsets], labels, options="COLZ", do_profile=False, text=text)


#### All clusters in DR 0.2 from GEN

In [ ]:
histo_resoCone_df = pd.DataFrame(index=tp_sets, columns=particles)
for tp_set in tp_sets:
    for particle in particles:
        histo_resoCone_df.loc[tp_set][particle] =  [histos.HistoSetReso('{}_{}'.format(tp_set, particle), sample.histo_file).hresoCone for sample in samples]
histo_resoCone_df

In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_ptRes for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_ptResp for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_n010 for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset = [histo.h_n025 for histo in histo_resoCone_df.loc[tp_set, particle]]
        drawSame(hByTPset, labels, norm=True, text=titles_df.loc[tp_set, particle])


We now compare the 2 defitnitions (highest pt and all clusters in cone) for the same PU point

In [ ]:
for particle in particles:
    for tp_set in ['DEF', 'DEF_em']:
        hByTPset_best = [histo.h_ptRes for histo in histo_reso_df.loc[tp_set, particle]]
        hByTPset_cone = [histo.h_ptRes for histo in histo_resoCone_df.loc[tp_set, particle]]

        drawSame([hByTPset_best[0], hByTPset_cone[0]], ['PU0 best', 'PU0 cone'], norm=True, text=titles_df.loc[tp_set, particle])
        drawSame([hByTPset_best[1], hByTPset_cone[1]], ['PU200 best', 'PU200 cone'], norm=True, text=titles_df.loc[tp_set, particle])


We now show the behavior of the pt residuals vs pt of the GEN particle

In [ ]:
for algo in algos:
    hByAlgo = [histo.h_ptResVpt for histo in histosAlgos_all.loc[algo].hresoCone]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True)
    #print hByAlgo[0].GetName()

In [ ]:
for algo in algos:
    hCl3D_ptResVpt_0 = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 1, 5) for histo in histosAlgos_all.loc[algo].hresoCone]
    for histo in hCl3D_ptResVpt_0:
        histo.SetTitle('3D cluster reso (GeV) for 1 < pt < 10 GeV')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hCl3D_ptResVpt_0, histosAlgos_all.loc[algo]['labels'], norm=True)


In [ ]:
for algo in algos:
    hCl3D_ptResVpt_1 = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 6, 10) for histo in histosAlgos_all.loc[algo].hresoCone]
    for histo in hCl3D_ptResVpt_1:
        histo.SetTitle('3D cluster reso (GeV) for 10 < pt < 20 GeV')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hCl3D_ptResVpt_1, histosAlgos_all.loc[algo]['labels'], norm=True)


In [ ]:
for algo in algos:
    hCl3D_ptResVpt_2 = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 11, 15) for histo in histosAlgos_all.loc[algo].hresoCone]
    for histo in hCl3D_ptResVpt_2:
        histo.SetTitle('3D cluster reso (GeV) for 20 < pt < 30 GeV')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hCl3D_ptResVpt_2, histosAlgos_all.loc[algo]['labels'], norm=True)


In [ ]:
for algo in algos:
    hCl3D_ptResVpt_3 = [histo.h_ptResVpt.ProjectionY(uuid.uuid4().hex[:6], 16, 20) for histo in histosAlgos_all.loc[algo].hresoCone]
    for histo in hCl3D_ptResVpt_3:
        histo.SetTitle('3D cluster reso (GeV) for 30 < pt < 40 GeV')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hCl3D_ptResVpt_3, histosAlgos_all.loc[algo]['labels'], norm=True)


In [ ]:
for algo in algos:
    hByAlgo = [histo.h_ptResVeta for histo in histosAlgos_all.loc[algo].hresoCone]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True)

In [ ]:
for algo in algos:
    hByAlgo = [histo.h_ptResVnclu for histo in histosAlgos_all.loc[algo].hresoCone]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True)

### Energy resolution


In [ ]:


for algo in algos:
    hByAlgo = [histo.h_energyRes for histo in histosAlgos_all.loc[algo].hreso]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSame(hByAlgo, histosAlgos_all.loc[algo]['labels'], norm=True)



In [ ]:

for s_id in range(0, len(samples)):
    labels_bySample = histosAlgos_all['labels'].apply(lambda x: x[s_id])
    hBySample = [histo.h_energyRes for histo in  histosAlgos_all['hreso'].apply(lambda x: x[s_id])]
    drawSame(hBySample, labels_bySample,'hist',logy=True)


## Postion resolution of 2D clusters

In [ ]:
for algo in algos:
    hByAlgo = [histo.h_xResVlayer for histo in histosAlgos_all.loc[algo].hreso2D]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True, miny=-2, maxy=2)

In [ ]:
for algo in algos:
    hByAlgo = [histo.h_xResVlayer.ProjectionY(uuid.uuid4().hex[:6], 1, 3) for histo in histosAlgos_all.loc[algo].hreso2D]
    for histo in hByAlgo:
        histo.SetTitle('2D cluster x reso (cm) for layer 1 and 3')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hByAlgo, histosAlgos_all.loc[algo]['labels'], norm=True, options='hist')


In [ ]:
for algo in algos:
    hByAlgo = [histo.h_xResVlayer.ProjectionY(uuid.uuid4().hex[:6], 12, 15) for histo in histosAlgos_all.loc[algo].hreso2D]
    for histo in hByAlgo:
        histo.SetTitle('2D cluster x reso (cm) for layer 13 and 15')
    #     normalize(hByAlgo, histosAlgos_all.loc[algo]['norm'])
    drawSame(hByAlgo, histosAlgos_all.loc[algo]['labels'], norm=True, options='hist')


In [ ]:
for algo in algos:
    hByAlgo = [histo.h_yResVlayer for histo in histosAlgos_all.loc[algo].hreso2D]
    #normalize(hclAll_pt35_layer, nevents_pt35)
    drawSeveral(hByAlgo, histosAlgos_all.loc[algo]['labels'], 'COLZ',do_profile=True, miny=-2, maxy=2)